# Analysis of the Ground State of a NSM Hamiltonian

We want to understand if the Quasiparticle decomposition [CITE HERE PAPERS] of the NSM Hamiltonian can be easily implemented. In order to do that, we want to look at the component of a ground state of a Nucleus using the NSM representation. In this case, since the configuration space is small, we are going to study a p-shell Nucleus.

#### Imports

In [6]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter
file_name='data/cki'
qq_filename='data/qq.p'
SPS=SingleParticleState(file_name=file_name)

#### Hyperparameters

In [15]:
nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

matrix_qq,_=get_twobody_nuclearshell_model(file_name=qq_filename)


Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:02<00:00,  5.94it/s]


#### Set up of the Quantum Objects and computation of the ground state

In [16]:


# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator

QQoperator=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
QQoperator.get_twobody_interaction(twobody_dict=matrix_qq)
QQoperator.get_hamiltonian()

# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))
#print('j_value=',J2Class.j_value(psi0))
print('deformation=',psi0.transpose().conjugate().dot(QQoperator.hamiltonian.dot(psi0)))




0it [00:00, ?it/s]

1256it [00:01, 766.74it/s]


size= 12 6
Computing the matrix, pls wait... (u_u) 



100%|██████████| 12/12 [00:02<00:00,  5.79it/s]
1424it [00:01, 727.71it/s]


[-30.29539461]
total_m= [0.]
deformation= [[-7.11447171]]


#### Analysis of the component of the Ground state

In [ ]:





# we need to get the argsort for
prob=(np.conj(psi0)*psi0)[:,0]

pr=np.sum(prob**2)
print('pr 8^Be=',pr,1/prob.shape[0],'\n')

plt.bar(np.arange(prob.shape[0]),prob)
plt.show()

print(prob.shape)
idx_sort=np.argsort(-1*prob[:])

cutoff_value=0.005
cutoff_prob=prob[prob>cutoff_value]


psi_cutoff=psi0.copy()
psi_cutoff[prob<cutoff_value]=0.
psi_cutoff=psi_cutoff/np.linalg.norm(psi_cutoff)
print(psi_cutoff.shape)
print(psi_cutoff.conj().T @ NSMHamiltonian.hamiltonian @ psi_cutoff)

psi_out=psi0.copy()
psi_out[prob>=cutoff_value]=0.
psi_out=psi_out/np.linalg.norm(psi_out)
print(psi_out.shape)
print(psi_out.conj().T @ NSMHamiltonian.hamiltonian @ psi_out)



plt.hist(prob,bins=40)
plt.show()

print(np.sum(cutoff_prob))
plt.show()


from itertools import combinations

couples_nn=[[0,3],[1,2],[4,5],[6,9],[7,8],[10,11],]
couples_np=[[0,9],[3,6],[1,8],[2,7],[4,11],[5,10]]

def new_base_computation(base:np.ndarray):
    
    indices=np.nonzero(base)[0]
    pairs = list(combinations(indices, 2))
    new_base=np.zeros(basis.shape[-1])
   
    print('base=',base,'\n')     
            
    list_of_token_indices=[]
    
    for i in range(len(couples_nn)):
        
        if base[couples_nn[i][0]]+base[couples_nn[i][1]]!=2 :
            continue
        else:
            new_base[i]+=1
            base[couples_nn[i][0]]=0
            base[couples_nn[i][1]]=0
    for i in range(len(couples_np)):
        if base[couples_np[i][0]]+base[couples_np[i][1]]!=2 :
            continue
        else:
            new_base[i+len(couples_nn)]+=1
            base[couples_np[i][0]]=0
            base[couples_np[i][1]]=0
    
    print(list_of_token_indices,new_base)
    exit
    return new_base
print(psi0.shape)
prob=0.
basis=NSMHamiltonian.basis.copy()
print(basis)
for i,base in enumerate(basis[:]):
    print(base,'auuu')
    new_base=new_base_computation(base)
    print(new_base)
    if np.sum(new_base)==(nparticles_a+nparticles_b)//2:
        prob+=psi0[i,0]*np.conjugate(psi0[i,0])

    
print('prob=',prob)
    


#### Comparison of different IPR computations

Imports

In [ ]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt

file_name='data/usdb.nat'
qq_filename='data/qq.sd'
SPS=SingleParticleState(file_name=file_name)

Data loading

In [ ]:
data=np.load('data/ipr_computation_pshell.npz')
### temporary
tf=20
time=np.linspace(0,tf,int(tf*10))
labels=data['labels']
iprs=data['ipr']
qqs=data['qq']
entanglement_pn=data['entropy_proton_neutron']
entanglement_initial=data['entropy_initial_rest']

plt.figure(figsize=(10,10))
for idx,ipr in enumerate(iprs):
    plt.plot(time,ipr,label=labels[idx])
    
plt.legend(fontsize=30)
plt.semilogy()
plt.xlabel(r'$t \omega$',fontsize=30)
plt.ylabel(r'$IPR(t)-\frac{1}{N}$',fontsize=30)
plt.tick_params(which='major',labelsize=30)
plt.tick_params(which='minor',labelsize=30)
plt.grid(True,which='minor')

plt.show()

plt.figure(figsize=(10,10))
for i in range(len(labels)):
    plt.plot(qqs[i],iprs[i],label=labels[i])
plt.legend(fontsize=20)
plt.show()


Analysis of entanglement

In [ ]:
plt.figure(figsize=(10,10))
for i in range(len(labels)):
    
    plt.plot(time,entanglement_pn[i]/np.log(2**6),label=labels[i])
plt.legend(fontsize=20)
plt.xlabel(r'$t \omega$',fontsize=30)
plt.ylabel(r'$S_{np}(t)/S_{max}$',fontsize=30)
plt.tick_params(labelsize=30)
plt.show()

#### QuasiParticleMapping

In [17]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)

print(QPC.couples)

#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles)
print(QPC.quasiparticle_basis)


[[0, 3], [1, 2], [4, 5], [6, 9], [7, 8], [10, 11], [0, 9], [1, 8], [2, 7], [3, 6], [4, 11], [5, 10]]
base= [1 1 0 0 0 0 0 0 1 1 0 0] 

qp base= [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
0 0
base= [1 1 0 0 0 0 0 0 0 1 0 1] 

base= [1 0 1 0 0 0 0 1 0 1 0 0] 

qp base= [0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
1 2
base= [1 0 1 0 0 0 0 0 1 0 0 1] 

base= [1 0 1 0 0 0 0 0 0 1 1 0] 

base= [1 0 0 1 0 0 1 0 0 1 0 0] 

qp base= [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
2 5
base= [1 0 0 1 0 0 0 1 1 0 0 0] 

qp base= [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
3 6
base= [1 0 0 1 0 0 0 1 0 0 0 1] 

base= [1 0 0 1 0 0 0 0 1 0 1 0] 

base= [1 0 0 1 0 0 0 0 0 0 1 1] 

qp base= [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
4 9
base= [1 0 0 0 1 0 0 0 1 1 0 0] 

base= [1 0 0 0 1 0 0 0 0 1 0 1] 

qp base= [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
5 11
base= [1 0 0 0 0 1 0 1 0 1 0 0] 

base= [1 0 0 0 0 1 0 0 1 0 0 1] 

base= [1 0 0 0 0 1 0 0 0 1 1 0] 

qp base= [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
6 14
base= [0 1 1 0 0 0 1 0 0 1 0 0

In [18]:

quasiparticle_hamiltonian=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
print(quasiparticle_hamiltonian)

values,psi=eigsh(quasiparticle_hamiltonian,k=1)

print(values,psi)

  (0, 0)	-6.582989999999999
  (0, 1)	-2.315995000000001
  (0, 2)	-1.052265
  (0, 3)	1.9530200000000002
  (0, 5)	-1.7096958900278656
  (0, 6)	2.054162795550007
  (0, 7)	1.9530199999999995
  (0, 8)	-1.0522650000000002
  (0, 10)	-0.9007550000000002
  (0, 11)	1.3652289845057242
  (0, 12)	-2.3986297010721485
  (1, 0)	-2.315995000000001
  (1, 1)	-8.705850000000002
  (1, 2)	-0.009165000000000756
  (1, 3)	-3.01445
  (1, 5)	2.054162795550007
  (1, 6)	-1.7096958900278656
  (1, 7)	-3.0144500000000014
  (1, 8)	-0.009165000000000534
  (1, 13)	-0.9007550000000002
  (1, 14)	1.3652289845057242
  (1, 15)	-2.3986297010721485
  (2, 0)	-1.052265
  (2, 1)	-0.009165000000000756
  (2, 2)	-15.405709999999996
  :	:
  (18, 18)	-4.514274999999993
  (18, 19)	1.043099999999999
  (18, 20)	-3.7638586855778717
  (19, 3)	-3.763858685577873
  (19, 8)	3.763858685577873
  (19, 11)	-2.6208249999999995
  (19, 12)	2.0813625000000004
  (19, 14)	2.0813625
  (19, 15)	-2.620825
  (19, 18)	1.043099999999999
  (19, 19)	-4.5142749